In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix
GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
LENGTH = 4.5  # [m]
WIDTH = 2.0  # [m]
BACKTOWHEEL = 1.0  # [m]
WHEEL_LEN = 0.3  # [m]
WHEEL_WIDTH = 0.2  # [m]
TREAD = 0.7  # [m]
WB = 2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True



        

def pi_2_pi(angle):
    while(angle > math.pi):
        angle = angle - 2.0 * math.pi

    while(angle < -math.pi):
        angle = angle + 2.0 * math.pi

    return angle





def plot_car(x, y, yaw, steer=0.0, cabcolor="-r", truckcolor="-k"):  # pragma: no cover

    outline = np.array([[-BACKTOWHEEL, (LENGTH - BACKTOWHEEL), (LENGTH - BACKTOWHEEL), -BACKTOWHEEL, -BACKTOWHEEL],
                        [WIDTH / 2, WIDTH / 2, - WIDTH / 2, -WIDTH / 2, WIDTH / 2]])

    fr_wheel = np.array([[WHEEL_LEN, -WHEEL_LEN, -WHEEL_LEN, WHEEL_LEN, WHEEL_LEN],
                         [-WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, WHEEL_WIDTH - TREAD, -WHEEL_WIDTH - TREAD]])

    rr_wheel = np.copy(fr_wheel)

    fl_wheel = np.copy(fr_wheel)
    fl_wheel[1, :] *= -1
    rl_wheel = np.copy(rr_wheel)
    rl_wheel[1, :] *= -1

    Rot1 = np.array([[math.cos(yaw), math.sin(yaw)],
                     [-math.sin(yaw), math.cos(yaw)]])
    Rot2 = np.array([[math.cos(steer), math.sin(steer)],
                     [-math.sin(steer), math.cos(steer)]])

    fr_wheel = (fr_wheel.T.dot(Rot2)).T
    fl_wheel = (fl_wheel.T.dot(Rot2)).T
    fr_wheel[0, :] += WB
    fl_wheel[0, :] += WB

    fr_wheel = (fr_wheel.T.dot(Rot1)).T
    fl_wheel = (fl_wheel.T.dot(Rot1)).T

    outline = (outline.T.dot(Rot1)).T
    rr_wheel = (rr_wheel.T.dot(Rot1)).T
    rl_wheel = (rl_wheel.T.dot(Rot1)).T

    outline[0, :] += x
    outline[1, :] += y
    fr_wheel[0, :] += x
    fr_wheel[1, :] += y
    rr_wheel[0, :] += x
    rr_wheel[1, :] += y
    fl_wheel[0, :] += x
    fl_wheel[1, :] += y
    rl_wheel[0, :] += x
    rl_wheel[1, :] += y

    plt.plot(np.array(outline[0, :]).flatten(),
             np.array(outline[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fr_wheel[0, :]).flatten(),
             np.array(fr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rr_wheel[0, :]).flatten(),
             np.array(rr_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(fl_wheel[0, :]).flatten(),
             np.array(fl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(np.array(rl_wheel[0, :]).flatten(),
             np.array(rl_wheel[1, :]).flatten(), truckcolor)
    plt.plot(x, y, "*")




def get_nparray_from_matrix(x):
    return np.array(x).flatten()


def get_closest_point(state, ccx, ccy, cyaw, pind):
    print(ccx,ccy)
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    print('printing d',d,len(d),type(d))
    mind = min(d)
    ind = d.index(mind) + pind
    print('ds',ind,mind)
    return ind, mind




def ccalc_ref_trajectory(state, cx, cy, cyaw, ck, sp, dl, pind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    ind, _ = get_closest_point(state, cx, cy, cyaw, pind)

    if pind >= ind:
        ind = pind

    xref[0, 0] = cx[ind]
    xref[1, 0] = cy[ind]
    xref[2, 0] = sp[ind]
    xref[3, 0] = cyaw[ind]
    dref[0, 0] = 0.0  # steer operational point should be 0

    S = 0.0
    vel=state.v
    for i in range(T + 1):
        S += abs(vel) * DT
        vel+=acc*DT
        dind = int(round(travel / dl))

        if (ind + dind) < ncourse:
            xref[0, i] = cx[ind + dind]
            xref[1, i] = cy[ind + dind]
            xref[2, i] = sp[ind + dind]
            xref[3, i] = cyaw[ind + dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = cx[ncourse - 1]
            xref[1, i] = cy[ncourse - 1]
            xref[2, i] = sp[ncourse - 1]
            xref[3, i] = cyaw[ncourse - 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False




def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile


def smooth_yaw(yaw):

    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw


def get_straight_course(dl):
    ax = [0.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0]
    ay = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course2(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_straight_course3(dl):
    ax = [0.0, -10.0, -20.0, -40.0, -50.0, -60.0, -70.0]
    ay = [0.0, -1.0, 1.0, 0.0, -1.0, 1.0, 0.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    cyaw = [i - math.pi for i in cyaw]

    return cx, cy, cyaw, ck


def get_forward_course(dl):
    ax = [0.0, 60.0, 125.0, 50.0, 75.0, 30.0, -10.0]
    ay = [0.0, 0.0, 50.0, 65.0, 30.0, 50.0, -20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)

    return cx, cy, cyaw, ck


def get_switch_back_course(dl):
    ax = [0.0, 30.0, 6.0, 20.0, 35.0]
    ay = [0.0, 0.0, 20.0, 35.0, 20.0]
    cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    ax = [35.0, 10.0, 0.0, 0.0]
    ay = [20.0, 30.0, 5.0, 0.0]
    cx2, cy2, cyaw2, ck2, s2 = cubic_spline_planner.calc_spline_course(
        ax, ay, ds=dl)
    cyaw2 = [i - math.pi for i in cyaw2]
    cx.extend(cx2)
    cy.extend(cy2)
    cyaw.extend(cyaw2)
    ck.extend(ck2)

    return cx, cy, cyaw, ck


def main():
    

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)




def main2():
    print(__file__ + " start!!")

    dl = 1.0  # course tick
    cx, cy, cyaw, ck = get_straight_course3(dl)

    sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

    initial_state = State(x=cx[0], y=cy[0], yaw=0.0, v=0.0)

    t, x, y, yaw, v, d, a = do_simulation(
        cx, cy, cyaw, ck, sp, dl, initial_state)

    if show_animation:  # pragma: no cover
        plt.close("all")
        plt.subplots()
        plt.plot(cx, cy, "-r", label="spline")
        plt.plot(x, y, "-g", label="tracking")
        plt.grid(True)
        plt.axis("equal")
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.legend()

        plt.subplots()
        plt.plot(t, v, "-r", label="speed")
        plt.grid(True)
        plt.xlabel("Time [s]")
        plt.ylabel("Speed [kmh]")

        plt.show()

In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")

    return oa, od, ox, oy, oyaw, ov

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = get_nparray_from_matrix(x.value[0, :])
        oy = get_nparray_from_matrix(x.value[1, :])
        ov = get_nparray_from_matrix(x.value[2, :])
        oyaw = get_nparray_from_matrix(x.value[3, :])
        oa = get_nparray_from_matrix(u.value[0, :])
        odelta = get_nparray_from_matrix(u.value[1, :])

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta):

        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = self.state.x + self.state.v * math.cos(self.state.yaw) * DT
        next_y = self.state.y + self.state.v * math.sin(self.state.yaw) * DT
        next_yaw = self.state.yaw + self.state.v / WB * math.tan(delta) * DT
        next_v = self.state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=20#2
    dy=10#1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [ ]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time
import cubic_spline_planner
def rad_to_deg():
    pass
def accl_to_gas(accl,vel):
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    #mass=.25
    power=4*mass*accl*(vel+accl/2)
    gas=power/ENGINE_POWER
    return gas

def env_step(env,action,show_plot=False):
   
    frame=np.zeros((100,200,3),dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    nst,rw,ter,info=envc.step(action) # take a random action
    frame[0:96,0:96,:]=np.array(nst)
    
    pos=envc.car.hull.position
    vel=np.sqrt(envc.car.hull.linearVelocity[0]**2+envc.car.hull.linearVelocity[1]**2)
    ang=envc.car.hull.angle
    if show_plot==True:
        plt.cla()
        img = cv2.putText(np.array(frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
        img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
        image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)
       
        plt.imshow(image)
        plt.show()
        plt.pause(0.0001)
        
    return [pos[0],pos[1],ang,vel],rw,ter,info
#envc.close()


envc=CarRacing()
def env_reset(envc):
    import pickle
    envc.reset()
    save=False
    if save==True:
        f=open('path-1','wb')
        pickle.dump(envc.track,f)
        f.close()
    f=open('path-1','rb')
    track=pickle.load(f)
    f.close()
    envc.track=track
    
    points=[[x,y] for a,b,x,y in envc.track]
    points=np.array(points)
    plt.plot(points[:,0],points[:,1])
    plt.show()
    return [env_step(envc,[0,0,0]),points]
    
#st,points=env_reset(envc)
'''
for i in range(100):
    st,rw,ter,info=env_step(envc,[0,1,0])
    time.sleep(0.1)
    #print(st)
    '''
#envc.close()



'\nfor i in range(100):\n    st,rw,ter,info=env_step(envc,[0,1,0])\n    time.sleep(0.1)\n    #print(st)\n    '

In [ ]:
envc.close()

In [ ]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
envc=CarRacing()
[init_state,traj_points]=env_reset(envc)
ix,iy,iyaw,ivel=init_state[0]
tx,ty=traj_points[:50,0],traj_points[:50,1]

cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
yaw0=cyaw[0]
cyaw=[yaw-yaw0 for yaw in cyaw]
#dcx=np.gradient(cx)
#dcy=np.gradient(cy)
#cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]
sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)


Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
print(state)

Track generation: 1288..1614 -> 326-tiles track


In [ ]:
def get_closest_point(state, ccx, ccy, cyaw, pind):
    #print(ccx,ccy,state.x,state.y)
    d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in zip(ccx,ccy)]
    #print('printing d',d,len(d),type(d))
    mind = min(d)
    ind = d.index(mind) + pind
    #print('ds',ind,mind)
    return ind, mind

def calc_ref_trajectory(state, ccx, ccy, ccyaw, ck, ssp, dl, ind):
    xref = np.zeros((NX, T + 1))
    dref = np.zeros((1, T + 1))
    ncourse = len(cx)

    #ind, _ = get_closest_point(state, ccx, ccy, ccyaw, pind)
    #if pind >= ind:
    #    ind = pind

    xref[0, 0] = ccx[0]
    xref[1, 0] = ccy[0]
    xref[2, 0] = ssp[0]
    xref[3, 0] = ccyaw[0]
    dref[0, 0] = 0.0  # steer operational point should be 0

    travel = 0.0

    for i in range(T + 1):
        travel += abs(state.v) * DT
        dind = int(round(travel / dl))

        if (dind) < len(ccx)-1:
            xref[0, i] = ccx[dind]
            xref[1, i] = ccy[dind]
            xref[2, i] = ssp[dind]
            xref[3, i] = ccyaw[dind]
            dref[0, i] = 0.0
        else:
            xref[0, i] = ccx[-1]
            xref[1, i] = ccy[-1]
            xref[2, i] = ssp[- 1]
            xref[3, i] = ccyaw[- 1]
            dref[0, i] = 0.0

    return xref, ind, dref

In [ ]:
# initial yaw compensation
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
Motion_Status.set_state(state)
goal = [cx[-1], cy[-1]]

In [ ]:
dl=1.0
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]
def get_raw_sensor_value():
    pass
_cx=cx[:10]
_cy=cy[:10]
_cyaw=cyaw[:10]
_sp=sp[:10]
#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
index=0
cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        print("x,y,yaw,v\n",state.x,state.y,state.yaw,state.v)
        
        index, _ = get_closest_point(state, _cx, _cy, cyaw, index)
        #if pind >= index:
        #    index = pind
        _cx=cx[index:index+15]
        _cy=cy[index:index+15]
        _cyaw=cyaw[index:index+15]
        _sp=sp[index:index+15]
        
        pind=index
        xref,_, dref = calc_ref_trajectory(state, _cx, _cy,_cyaw, ck, _sp, dl, index)
        print('xref...............\n',xref,'\n')
        x0 = [state.x, state.y, state.v, state.yaw]  # current state

        oa, odelta, ox, oy, oyaw, ov = iterative_linear_mpc_control(
            xref, Motion_Status, dref, oa, odelta,x0)

        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        gas=accl_to_gas(accl,state.v)
        steer=di
        print('accl,gas,steer\n',accl,gas,steer)
        nst,rw,ter,info=env_step(envc,[steer,gas,0])
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        #next_state = Motion_Status.get_next_state(ai, di)
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        if check_goal(state, goal, index, len(cx)):
            print("Goal")
            break

        if show_animation:  # pragma: no cover
            plt.cla()
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[index], cy[index], "xg", label="target")
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
                      + '\nyaw'+str(state.yaw)
                      + '\nxref_yaw'+str(xref[3][0])
                      + '\nsteer'+str(steer))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


x,y,yaw,v
 218.73988342285156 -51.4577751159668 -0.11570970714092255 9.474043820239349e-07
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -0.9999999998579724 0.0002962957347873129 -0.22547269665856187
214.73988342285156 -49.4577751159668
x,y,yaw,v
 218.7399139404297 -51.45763397216797 -0.1157444640994072 0.007187192386544108
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.

x,y,yaw,v
 218.74073791503906 -51.44733428955078 -0.11550544947385788 0.07188222074047172
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.02288844e-10  1.02288844e-10  1.02288844e-10  1.02288844e-10
   1.02288844e-10  1.02288844e-10]] 

accl,gas,steer
 -0.9999999996494565 0.0002536994245535897 -0.09069621348063482
214.74073791503906 -49.44733428955078
x,y,yaw,v
 218.74085998535156 -51.44580078125 -0.11547495424747467 0.07684050612616675
xref...............
 [[ 2.19738705e+02  2.19738705e+02  2.19738705e+02  2.19738705e+02
   2.19738705e+02  2.19738705e+02]
 [-5.20321233e+01 -5.20321233e+01 -5.20321233e+01 -5.20321233e+01
  -5.20321233e+01 -5.20321233e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77

x,y,yaw,v
 218.7433624267578 -51.42018508911133 -0.11519501358270645 0.1316285957701029
xref...............
 [[ 2.19840098e+02  2.19840098e+02  2.19840098e+02  2.19840098e+02
   2.19840098e+02  2.19840098e+02]
 [-5.10372768e+01 -5.10372768e+01 -5.10372768e+01 -5.10372768e+01
  -5.10372768e+01 -5.10372768e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64798841e-10  1.64798841e-10  1.64798841e-10  1.64798841e-10
   1.64798841e-10  1.64798841e-10]] 

accl,gas,steer
 -0.9999999999054163 0.0002182941654208968 -0.006274973247375112
214.7433624267578 -49.42018508911133
x,y,yaw,v
 218.74366760253906 -51.417484283447266 -0.11518160253763199 0.13582312667246407
xref...............
 [[ 2.19840098e+02  2.19840098e+02  2.19840098e+02  2.19840098e+02
   2.19840098e+02  2.19840098e+02]
 [-5.10372768e+01 -5.10372768e+01 -5.10372768e+01 -5.10372768e+01
  -5.10372768e+01 -5.10372768e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.

x,y,yaw,v
 218.74859619140625 -51.37904357910156 -0.11538983136415482 0.182052934396773
xref...............
 [[ 2.19840098e+02  2.19840098e+02  2.19840098e+02  2.19840098e+02
   2.19840098e+02  2.19840098e+02]
 [-5.10372768e+01 -5.10372768e+01 -5.10372768e+01 -5.10372768e+01
  -5.10372768e+01 -5.10372768e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64798841e-10  1.64798841e-10  1.64798841e-10  1.64798841e-10
   1.64798841e-10  1.64798841e-10]] 

accl,gas,steer
 -0.9999999999006958 0.0001884130758648897 0.06730283328810595
214.74859619140625 -49.37904357910156
x,y,yaw,v
 218.74913024902344 -51.375370025634766 -0.1154448464512825 0.1855833094656543
xref...............
 [[ 2.19840098e+02  2.19840098e+02  2.19840098e+02  2.19840098e+02
   2.19840098e+02  2.19840098e+02]
 [-5.10372768e+01 -5.10372768e+01 -5.10372768e+01 -5.10372768e+01
  -5.10372768e+01 -5.10372768e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.777

x,y,yaw,v
 218.75717163085938 -51.3263053894043 -0.11671178042888641 0.22443622940870947
xref...............
 [[ 2.19840098e+02  2.19840098e+02  2.19840098e+02  2.19840098e+02
   2.19840098e+02  2.19840098e+02]
 [-5.10372768e+01 -5.10372768e+01 -5.10372768e+01 -5.10372768e+01
  -5.10372768e+01 -5.10372768e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 1.64798841e-10  1.64798841e-10  1.64798841e-10  1.64798841e-10
   1.64798841e-10  1.64798841e-10]] 

accl,gas,steer
 -0.9999999993620913 0.00016329704894610465 0.1385667984331704
214.75717163085938 -49.3263053894043
x,y,yaw,v
 218.7579803466797 -51.32183074951172 -0.11687351763248444 0.22739822457823738
xref...............
 [[ 2.19840098e+02  2.19840098e+02  2.19840098e+02  2.19840098e+02
   2.19840098e+02  2.19840098e+02]
 [-5.10372768e+01 -5.10372768e+01 -5.10372768e+01 -5.10372768e+01
  -5.10372768e+01 -5.10372768e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.777

...

accl,gas,steer
 -0.9999992660898678 9.38807066206932e-06 0.748954818299909
223.41136169433594 -43.68388748168945
x,y,yaw,v
 227.42100524902344 -45.685001373291016 -1.4769883155822754 0.4841572939040797
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 7.10340009e-10  7.10340009e-10  7.10340009e-10  7.10340009e-10
   7.10340009e-10  1.64798908e-09]] 

accl,gas,steer
 -0.999999328491042 9.388065009272719e-06 0.7487404274763308
223.42100524902344 -43.685001373291016
x,y,yaw,v
 227.43064880371094 -45.68613052368164 -1.4781821966171265 0.4841573061935613
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980

223.5365753173828 -43.69929504394531
x,y,yaw,v
 227.5461883544922 -45.7005615234375 -1.492508888244629 0.4841574739043225
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 7.10340009e-10  7.10340009e-10  7.10340009e-10  7.10340009e-10
   7.10340009e-10  1.64798908e-09]] 

accl,gas,steer
 -0.9999998701323142 9.388123913724922e-06 0.744008146994813
223.5461883544922 -43.7005615234375
x,y,yaw,v
 227.55581665039062 -45.701839447021484 -1.49370276927948 0.4841574889671463
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.7777777

accl,gas,steer
 -0.9999998188833004 9.388047899207423e-06 0.7378196488801854
223.67112731933594 -43.718055725097656
x,y,yaw,v
 227.68072509765625 -45.719482421875 -1.5092231035232544 0.4841576369799146
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 7.10340009e-10  7.10340009e-10  7.10340009e-10  7.10340009e-10
   7.10340009e-10  1.64798908e-09]] 

accl,gas,steer
 -0.9999991038013284 9.387793021183106e-06 0.7367844256565304
223.68072509765625 -43.719482421875
x,y,yaw,v
 227.69033813476562 -45.720916748046875 -1.5104169845581055 0.4841576064683331
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e

223.7957763671875 -43.737483978271484
x,y,yaw,v
 227.8053436279297 -45.73905944824219 -1.5247435569763184 0.48415774577595605
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 7.10340009e-10  7.10340009e-10  7.10340009e-10  7.10340009e-10
   7.10340009e-10  1.64798908e-09]] 

accl,gas,steer
 -0.9999993131707011 9.387792550344306e-06 0.7311238594588242
223.8053436279297 -43.73905944824219
x,y,yaw,v
 227.81491088867188 -45.74064636230469 -1.5259374380111694 0.4841577362092258
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.

223.9200897216797 -43.75885009765625
x,y,yaw,v
 227.9296417236328 -45.76057434082031 -1.5402638912200928 0.48415779691167726
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.77777778e+00 -2.77777778e+00 -2.77777778e+00
  -2.77777778e+00 -2.77777778e+00]
 [ 7.10340009e-10  7.10340009e-10  7.10340009e-10  7.10340009e-10
   7.10340009e-10  1.64798908e-09]] 

accl,gas,steer
 -0.9999995127063485 9.38782324255449e-06 0.7254547185024939
223.9296417236328 -43.76057434082031
x,y,yaw,v
 227.93917846679688 -45.762306213378906 -1.5414578914642334 0.48415788263636655
xref...............
 [[ 2.20448454e+02  2.20448454e+02  2.20448454e+02  2.20448454e+02
   2.20448454e+02  2.20549847e+02]
 [-4.50681980e+01 -4.50681980e+01 -4.50681980e+01 -4.50681980e+01
  -4.50681980e+01 -4.40733515e+01]
 [-2.77777778e+00 -2.

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cyaw